# **YOLO object detection with OpenCV**
![대체 텍스트](https://s3-us-west-2.amazonaws.com/static.pyimagesearch.com/opencv-yolo/yolo_car_chase_01_output.gif)

What is the YOLO object detector?

![대체 텍스트](http://media5.datahacker.rs/2019/04/yolo_diagram-1024x556.jpg)

We can say that the basic idea of the Yolo algorithm is applying both the image classification and localization algorithm on each of nine grid cells. 
![대체 텍스트](http://media5.datahacker.rs/2018/11/025_02.png)



# Data Load of image, video and model

In [ ]:
%%shell
mkdir objectDetection; cd objectDetection
mkdir model; cd model
curl -LJO https://github.com/phoebe81/DetectionWithOpenCV/raw/master/Model/coco.names
curl -LJO https://github.com/phoebe81/DetectionWithOpenCV/raw/master/Model/yolov3.cfg
curl -LJO https://github.com/phoebe81/DetectionWithOpenCV/raw/master/Model/yolov3.weights
cd ..; mkdir images; cd images
curl -LJO https://github.com/phoebe81/DetectionWithOpenCV/raw/master/Images/example_01.jpg
curl -LJO https://github.com/phoebe81/DetectionWithOpenCV/raw/master/Images/example_02.jpg
curl -LJO https://github.com/phoebe81/DetectionWithOpenCV/raw/master/Images/example_03.jpg
curl -LJO https://github.com/phoebe81/DetectionWithOpenCV/raw/master/Images/example_04.jpg
curl -LJO https://github.com/phoebe81/DetectionWithOpenCV/raw/master/Images/example_05.jpg
curl -LJO https://github.com/phoebe81/DetectionWithOpenCV/raw/master/Images/example_06.jpg

In [ ]:
# import the necessary packages
import numpy as np
import cv2
from google.colab.patches import cv2_imshow
import os

In [ ]:
# Root directory of the project
ROOT_DIR = os.path.abspath("./objectDetection/")
# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "model")
# Directory of images/video to run detection on
IMAGE_DIR = os.path.join(ROOT_DIR, "images")

weightsPath = os.path.sep.join([MODEL_DIR, "yolov3.weights"])
configPath = os.path.sep.join([MODEL_DIR, "yolov3.cfg"])
objectNet = cv2.dnn.readNetFromDarknet(configPath, weightsPath)

In [ ]:
# determine only the *output* layer names that we need from YOLO
ln = objectNet.getLayerNames()
ln = [ln[i[0] - 1] for i in objectNet.getUnconnectedOutLayers()]

In [ ]:
confThreshold  = 0.3
nmsThreshold = 0.3

In [ ]:
# load the COCO class labels our YOLO model was trained on
labelsPath = os.path.sep.join([MODEL_DIR, "coco.names"])
LABELS = open(labelsPath).read().strip().split("\n")
np.random.seed(42)
COLORS = np.random.randint(0, 255, size=(len(LABELS), 3),dtype="uint8")

# **1**.Detection model demo by picture

In [ ]:
# load the input image
image_list = []
layerOutputs_list = []
for file in os.listdir(IMAGE_DIR):
	file_path = os.path.join(IMAGE_DIR, file)
	if(os.path.isfile(file_path)): 
		image = cv2.imread(file_path)
		blob = cv2.dnn.blobFromImage(image, 1 / 255.0, (416, 416), swapRB=True, crop=False)
		objectNet.setInput(blob)	
		layerOutputs = objectNet.forward(ln)  
		image_list.append(image)   
		layerOutputs_list.append(layerOutputs)

In [ ]:
for i, layerOutputs in enumerate(layerOutputs_list):
	boxes = []
	confidences = []
	classIDs = []
	(H, W) = image_list[i].shape[:2]
	image = image_list[i]
	for output in layerOutputs:		
		for detection in output:
			scores = detection[5:]		
			classID = np.argmax(scores)
			confidence = scores[classID]

			if confidence > confThreshold:
				box = detection[0:4] * np.array([W, H, W, H])
				(centerX, centerY, width, height) = box.astype("int")

				x = int(centerX - (width / 2))
				y = int(centerY - (height / 2))

				boxes.append([x, y, int(width), int(height)])
				confidences.append(float(confidence))
				classIDs.append(classID)
	# apply non-maxima suppression to suppress weak, overlapping bounding boxes
	idxs = cv2.dnn.NMSBoxes(boxes, confidences, confThreshold, nmsThreshold)

	# ensure at least one detection exists
	if len(idxs) > 0:
		for i in idxs.flatten():
			(x, y) = (boxes[i][0], boxes[i][1])
			(w, h) = (boxes[i][2], boxes[i][3])

			color = [int(c) for c in COLORS[classIDs[i]]]
			cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)
			text = "{}: {:.4f}".format(LABELS[classIDs[i]], confidences[i])
			cv2.putText(image, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

In [ ]:
# show the output image
for image in image_list:
  cv2_imshow(image)

# **2**.Detection model demo by video

In [ ]:
%%shell
cd objectDetection
mkdir videos; cd videos
curl -LJO https://github.com/phoebe81/DetectionWithOpenCV/raw/master/Videos/car_chase_01.mp4
cd videos; mkdir save

In [ ]:
VIDEO_DIR = os.path.join(ROOT_DIR, "videos")
VIDEO_SAVE_DIR = os.path.join(VIDEO_DIR, "save")

## Processing Funtions

In [ ]:
import base64, logging
import numpy as np
from PIL import Image
from io import BytesIO

def data_uri_to_img(uri):
  try:
    image = base64.b64decode(uri.split(',')[1], validate=True)
    # make the binary image, a PIL image
    image = Image.open(BytesIO(image))
    # convert to numpy array
    image = np.array(image, dtype=np.uint8); 
    return image
  except Exception as e:
    logging.exception(e);print('\n')
    return None

def video_to_data_url(filename):
    ext = filename.split('.')[-1]
    prefix = 'data:video/{};base64,'.format(ext)
    with open(filename, 'rb') as f:
        vidoe = f.read()
    return prefix + base64.b64encode(vidoe).decode()

In [ ]:
def detect_and_predict_object(frame):
	blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416), swapRB=True, crop=False)
	objectNet.setInput(blob)
	layerOutputs = objectNet.forward(ln)
	
	boxes = []
	confidences = []
	classIDs = []
	for output in layerOutputs:		
		for detection in output:
			scores = detection[5:]		
			classID = np.argmax(scores)
			confidence = scores[classID]

			if confidence > confThreshold:
				box = detection[0:4] * np.array([frame.shape[1], frame.shape[0], frame.shape[1], frame.shape[0]])
				(centerX, centerY, width, height) = box.astype("int")

				x = int(centerX - (width / 2))
				y = int(centerY - (height / 2))

				boxes.append([x, y, int(width), int(height)])
				confidences.append(float(confidence))
				classIDs.append(classID)
	# apply non-maxima suppression to suppress weak, overlapping bounding boxes
	idxs = cv2.dnn.NMSBoxes(boxes, confidences, confThreshold, nmsThreshold)

	# ensure at least one detection exists
	if len(idxs) > 0:
		for i in idxs.flatten():
			(x, y) = (boxes[i][0], boxes[i][1])
			(w, h) = (boxes[i][2], boxes[i][3])

			color = [int(c) for c in COLORS[classIDs[i]]]
			cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
			text = "{}: {:.4f}".format(LABELS[classIDs[i]], confidences[i])
			cv2.putText(frame, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)


## Videdo Capture
Using a webcam to capture images for processing on the runtime.
Source: https://colab.research.google.com/notebooks/snippets/advanced_outputs.ipynb#scrollTo=2viqYx97hPMi

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

# playing webcam or video with javascript
def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''     
    async function takePhoto(filename, quality) {
                  
      const div = document.createElement('div');
      document.body.appendChild(div);

      const exit = document.createElement('button');
      exit.textContent = 'Exit';
      div.appendChild(exit);

      const video = document.createElement('video');
      video.style.display = 'block';
           
      if('photo.jpg' == filename){
        const stream = await navigator.mediaDevices.getUserMedia({video: true}); 
        video.srcObject = stream;   
      }else{
        video.src = filename;
        video.type="video/mp4"
      }
      await video.play();  
      div.appendChild(video);       
                   

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);
      
      let jsLog = function(abc) {
        document.querySelector("#output-area").appendChild(document.createTextNode(`${abc}... `));
      }

      // when Exit button is clicked.   
      var isOpened = true; 
      var exitPromise = new Promise((resolve) => {exit.onclick = resolve});   
      exitPromise.then(()=>{isOpened = false; stream.getVideoTracks()[0].stop();});
      
      //when end of video
      var endPromise = new Promise((resolve) => {video.onended = resolve});   
      endPromise.then(()=>{isOpened = false; video.stop();});

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;

      for (let i = 0; isOpened; i++) {
        canvas.getContext("2d").clearRect(0, 0, canvas.width, canvas.height);
        canvas.getContext('2d').drawImage(video, 0, 0);
        img = canvas.toDataURL('image/jpeg', quality);

        // jsLog(i + "sending");
        // Call a python function and send this image
        google.colab.kernel.invokeFunction('notebook.run_objectDetection', [img], {});
        // jsLog(i + "SENT");

        // wait for X miliseconds second, before next capture
        await new Promise(resolve => setTimeout(resolve, 250));        
      }       
      div.remove();      
    }    
    ''')  
  # make the provided HTML, part of the cell
  display(js)
  #call the takePhoto() JavaScript function
  eval_js('takePhoto({},{})'.format("'"+filename+"'", quality)) 

In [ ]:
from google.colab import output
frame_count = 0
writer = None

# InvokeFunction
# takes the numpy image and runs detection, then shows the results by visualizing
def run_objectDetection(uri): 
  global frame_count, writer

  image = data_uri_to_img(uri)     
  if writer is None:		
      fourcc = cv2.VideoWriter_fourcc(*'DIVX')  
      writer = cv2.VideoWriter(outVideo, fourcc, 2, (image.shape[1], image.shape[0]), True)
  try:        
    detect_and_predict_object(image)

    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 
    frame_count+=1    
    name = '{0}.jpg'.format(frame_count)
    name = os.path.join(VIDEO_SAVE_DIR, name)
    cv2.imwrite(name, image)

    if writer is not None:
      writer.write(image)
  except Exception as e:
    logging.exception(e)
    print('\n')

# register this function, so JS code could call this
output.register_callback('notebook.run_objectDetection', run_faceDetection)

## Apply Detection model

In [ ]:
 %%shell
 cd faceDection
 rm ./videos/save/* 
 rm ./videos/out.avi

In [ ]:
inVideo = os.path.join(VIDEO_DIR, "car_chase_01.mp4")
outVideo= os.path.join(VIDEO_DIR, "out.avi")

In [ ]:
frame_count = 0
data_url = video_to_data_url(inVideo)
try: 
  # put the JS code in cell and run it
  take_photo()  
  if writer is not None:
    writer.release()   
except Exception as e:
  logging.exception(e)
  print('\n')

writer = None

## Downlod  to our local machine

In [ ]:
from google.colab import files
files.download(outVideo)